In [83]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from openai import OpenAI
import os
import re

In [45]:
df = pd.read_csv('CausalNewsCorpus-Master/data/V2/train_subtask2_grouped.csv')


In [46]:
df.head()

,corpus,doc_id,sent_id,eg_id,index,text,causal_text_w_pairs,num_rs
0,cnc,train_01_0,892,0,cnc_train_01_0_892_0,"The State alleged they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016 , allegedly over the allocation of low cost ( RDP ) houses at Marikana West Extension 2 .","['The State alleged <ARG1>they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016</ARG1> , allegedly <SIG0>over</SIG0> <ARG0>the allocation of low cost ( RDP ) houses at Marikana West Extension 2</ARG0> .']",1
1,cnc,train_01_1,2714,0,cnc_train_01_1_2714_0,"Chale was allegedly chased by a group of about 30 people and was hacked to death with pangas , axes and spears .",[],0
2,cnc,train_01_10,2619,0,cnc_train_01_10_2619_0,The farmworkers ' strike resumed on Tuesday when their demands were not met .,"[""<ARG1>The farmworkers ' strike resumed on Tuesday</ARG1> when <ARG0>their demands were not met</ARG0> .""]",1
3,cnc,train_01_100,2680,0,cnc_train_01_100_2680_0,"Demonstrators have filed for a permit to hold a rally on Saturday in Yuen Long , the district on the outskirts of Hong Kong where dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods , leaving at least 45 people in hospital .","['<ARG1>Demonstrators have filed for a permit</ARG1> <ARG0><SIG0>to</SIG0> hold a rally on Saturday in Yuen Long</ARG0> , the district on the outskirts of Hong Kong where dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods , leaving at least 45 people in hospital .', 'Demonstrators have filed for a permit to hold a rally on Saturday in Yuen Long , the district on the outskirts of Hong Kong where <ARG0>dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods</ARG0> , <ARG1><SIG0>leaving</SIG0> at least 45 people in hospital</ARG1> .']",2
4,cnc,train_01_101,3090,0,cnc_train_01_101_3090_0,"Footage of the attack , which included a pregnant woman being hit , protesters being punched and kneed , and commuters screaming and crying while trying to shield themselves , emerged on Sunday night , fuelling further political unrest as demonstrators , opposition lawmakers and others demanded answers from authorities for failing to stop the violence .","['<ARG0>Footage of the attack , which included a pregnant woman being hit , protesters being punched and kneed , and commuters screaming and crying while trying to shield themselves , emerged on Sunday night</ARG0> , <ARG1><SIG0>fuelling</SIG0> further political unrest</ARG1> as demonstrators , opposition lawmakers and others demanded answers from authorities for failing to stop the violence .', 'Footage of the attack , which included a pregnant woman being hit , protesters being punched and kneed , and commuters screaming and crying while trying to shield themselves , emerged on Sunday night , fuelling further political unrest as <ARG1>demonstrators , opposition lawmakers and others demanded answers from authorities</ARG1> <SIG0>for</SIG0> <ARG0>failing to stop the violence</ARG0> .']",2


In [47]:
pd.melt(df)

,variable,value
0,corpus,cnc
1,corpus,cnc
2,corpus,cnc
3,corpus,cnc
4,corpus,cnc
...,...,...
24595,num_rs,2
24596,num_rs,0
24597,num_rs,0
24598,num_rs,1


In [48]:
df.head()

,corpus,doc_id,sent_id,eg_id,index,text,causal_text_w_pairs,num_rs
0,cnc,train_01_0,892,0,cnc_train_01_0_892_0,"The State alleged they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016 , allegedly over the allocation of low cost ( RDP ) houses at Marikana West Extension 2 .","['The State alleged <ARG1>they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016</ARG1> , allegedly <SIG0>over</SIG0> <ARG0>the allocation of low cost ( RDP ) houses at Marikana West Extension 2</ARG0> .']",1
1,cnc,train_01_1,2714,0,cnc_train_01_1_2714_0,"Chale was allegedly chased by a group of about 30 people and was hacked to death with pangas , axes and spears .",[],0
2,cnc,train_01_10,2619,0,cnc_train_01_10_2619_0,The farmworkers ' strike resumed on Tuesday when their demands were not met .,"[""<ARG1>The farmworkers ' strike resumed on Tuesday</ARG1> when <ARG0>their demands were not met</ARG0> .""]",1
3,cnc,train_01_100,2680,0,cnc_train_01_100_2680_0,"Demonstrators have filed for a permit to hold a rally on Saturday in Yuen Long , the district on the outskirts of Hong Kong where dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods , leaving at least 45 people in hospital .","['<ARG1>Demonstrators have filed for a permit</ARG1> <ARG0><SIG0>to</SIG0> hold a rally on Saturday in Yuen Long</ARG0> , the district on the outskirts of Hong Kong where dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods , leaving at least 45 people in hospital .', 'Demonstrators have filed for a permit to hold a rally on Saturday in Yuen Long , the district on the outskirts of Hong Kong where <ARG0>dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods</ARG0> , <ARG1><SIG0>leaving</SIG0> at least 45 people in hospital</ARG1> .']",2
4,cnc,train_01_101,3090,0,cnc_train_01_101_3090_0,"Footage of the attack , which included a pregnant woman being hit , protesters being punched and kneed , and commuters screaming and crying while trying to shield themselves , emerged on Sunday night , fuelling further political unrest as demonstrators , opposition lawmakers and others demanded answers from authorities for failing to stop the violence .","['<ARG0>Footage of the attack , which included a pregnant woman being hit , protesters being punched and kneed , and commuters screaming and crying while trying to shield themselves , emerged on Sunday night</ARG0> , <ARG1><SIG0>fuelling</SIG0> further political unrest</ARG1> as demonstrators , opposition lawmakers and others demanded answers from authorities for failing to stop the violence .', 'Footage of the attack , which included a pregnant woman being hit , protesters being punched and kneed , and commuters screaming and crying while trying to shield themselves , emerged on Sunday night , fuelling further political unrest as <ARG1>demonstrators , opposition lawmakers and others demanded answers from authorities</ARG1> <SIG0>for</SIG0> <ARG0>failing to stop the violence</ARG0> .']",2


In [49]:
pd.set_option('display.max_colwidth', None)
print(df.head()['text'])

0                                                                                                                                                                    The State alleged they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016 , allegedly over the allocation of low cost ( RDP ) houses at Marikana West Extension 2 .
1                                                                                                                                                                                                                                                       Chale was allegedly chased by a group of about 30 people and was hacked to death with pangas , axes and spears .
2                                                                                                                                                                                                                                                                                     

In [50]:
prompt = "Introduction: I'm seeking data augmentation for causal sentences, where each sentence comprises a cause, effect, and signal./n Please adhere to the following format:/n Cause: The reason for an event happening, to be enclosed between <ARG0> and </ARG0>./n Effect: The event that occurs due to the cause, to be enclosed between <ARG1> and </ARG1>./n Signal: Words that transition the cause to the effect, to be enclosed between <SIG0> and </SIG0>./n for example:/n  1. <ARG0>The heavy rainstorm</ARG0> in the evening <sig0>caused</sig0> <ARG1>widespread flooding in the city</ARG1>./n 2. After the long hours of practice, <ARG0>the team's dedication</ARG0> <sig0>led to</sig0> <ARG1>a remarkable improvement in their performance</ARG1>./n 3. <ARG0>The sudden power outage</ARG0> during the storm <sig0>resulted in</sig0> <ARG1>a disruption of the entire communication system</ARG1>./n 4. (Amelia's convincing arguments <sig0>played a crucial role in</sig0> <ARG0>convincing the board</ARG0> <ARG1>to approve the new project proposal</ARG1>./n  Augment the following sentences using the above format :/n The State alleged they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016 , allegedly over the allocation of low cost ( RDP ) houses at Marikana West Extension 2 ./n Chale was allegedly chased by a group of about 30 people and was hacked to death with pangas , axes and spears ./n The farmworkers ' strike resumed on Tuesday when their demands were not met ."

In [14]:
#add in your own api key below
OpenAI.api_key = ''
client = OpenAI(api_key = '')
s = ''
stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}],
    stream=True,
)
for part in stream:
    s += (part.choices[0].delta.content or "")

In [15]:
print(s)

1. <ARG0>The State</ARG0> alleged they hacked <ARG0>Sabata Petros Chale</ARG0>, 39, to death <SIG0>in Marikana West</SIG0>, on December 8, 2016, allegedly over <SIG0>the allocation of low cost (RDP) houses at Marikana West Extension 2</SIG0>.
2. <SIG0>Chale was allegedly chased by a group of about 30 people</SIG0> and <SIG0>was hacked to death with pangas, axes, and spears</SIG0>.
3. <ARG0>The farmworkers' strike</ARG0> <SIG0>resumed on Tuesday</SIG0> when <ARG0>their demands were not met</ARG0>.


In [16]:
sentences = s.split('\n')

# Removing empty strings from the list
sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

# Displaying the result
print(sentences[0])

1. <ARG0>The State</ARG0> alleged they hacked <ARG0>Sabata Petros Chale</ARG0>, 39, to death <SIG0>in Marikana West</SIG0>, on December 8, 2016, allegedly over <SIG0>the allocation of low cost (RDP) houses at Marikana West Extension 2</SIG0>.


In [17]:
modified_sentences = [sentence[4:] for sentence in sentences]

# Displaying the modified sentences
for modified_sentence in modified_sentences:
    print(modified_sentence)

ARG0>The State</ARG0> alleged they hacked <ARG0>Sabata Petros Chale</ARG0>, 39, to death <SIG0>in Marikana West</SIG0>, on December 8, 2016, allegedly over <SIG0>the allocation of low cost (RDP) houses at Marikana West Extension 2</SIG0>.
SIG0>Chale was allegedly chased by a group of about 30 people</SIG0> and <SIG0>was hacked to death with pangas, axes, and spears</SIG0>.
ARG0>The farmworkers' strike</ARG0> <SIG0>resumed on Tuesday</SIG0> when <ARG0>their demands were not met</ARG0>.


In [18]:
def pro_sen(s):
    sentences = s.split('\n')
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    modified_sentences = [sentence[3:] for sentence in sentences]
    return modified_sentences

In [19]:
print(pro_sen(s))

['<ARG0>The State</ARG0> alleged they hacked <ARG0>Sabata Petros Chale</ARG0>, 39, to death <SIG0>in Marikana West</SIG0>, on December 8, 2016, allegedly over <SIG0>the allocation of low cost (RDP) houses at Marikana West Extension 2</SIG0>.', '<SIG0>Chale was allegedly chased by a group of about 30 people</SIG0> and <SIG0>was hacked to death with pangas, axes, and spears</SIG0>.', "<ARG0>The farmworkers' strike</ARG0> <SIG0>resumed on Tuesday</SIG0> when <ARG0>their demands were not met</ARG0>."]


In [51]:
texts = df['text']

In [52]:
h_text = texts.head(10)

In [53]:
h_text

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             The State alleged they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016 , allegedly over the allocation of low cost ( RDP ) houses at Marikana West Extension 2 .
1                                                                                                                                                                                                                                                                                                     

In [23]:
combined_text = '\n\n'.join(h_text)

# Display the result
print(combined_text)

The State alleged they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016 , allegedly over the allocation of low cost ( RDP ) houses at Marikana West Extension 2 .

Chale was allegedly chased by a group of about 30 people and was hacked to death with pangas , axes and spears .

The farmworkers ' strike resumed on Tuesday when their demands were not met .

Demonstrators have filed for a permit to hold a rally on Saturday in Yuen Long , the district on the outskirts of Hong Kong where dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods , leaving at least 45 people in hospital .

Footage of the attack , which included a pregnant woman being hit , protesters being punched and kneed , and commuters screaming and crying while trying to shield themselves , emerged on Sunday night , fuelling further political unrest as demonstrators , opposition lawmakers and others demanded answers from authorities for failing to stop 

In [59]:
def create_subdataset(texts, flag=False, prompt = "Introduction: I'm seeking data augmentation for causal sentences, where each sentence comprises a cause, effect, and signal./n Please adhere to the following format:/n Cause: The reason for an event happening, to be enclosed between <ARG0> and </ARG0>./n Effect: The event that occurs due to the cause, to be enclosed between <ARG1> and </ARG1>./n Signal: Words that transition the cause to the effect, to be enclosed between <SIG0> and </SIG0>./n for example:/n  1. <ARG0>The heavy rainstorm</ARG0> in the evening <sig0>caused</sig0> <ARG1>widespread flooding in the city</ARG1>./n 2. After the long hours of practice, <ARG0>the team's dedication</ARG0> <sig0>led to</sig0> <ARG1>a remarkable improvement in their performance</ARG1>./n 3. <ARG0>The sudden power outage</ARG0> during the storm <sig0>resulted in</sig0> <ARG1>a disruption of the entire communication system</ARG1>./n 4. (Amelia's convincing arguments <sig0>played a crucial role in</sig0> <ARG0>convincing the board</ARG0> <ARG1>to approve the new project proposal</ARG1>./n  Augment the following sentences using the above format\n"):
    combined_text = '\n\n'.join(texts)
    if flag == True:
        prompt = prompt + texts
    else:
        prompt = prompt + combined_text
    #add in your own api key below
    client = OpenAI(api_key = '')
    s = ''
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    for part in stream:
        s += (part.choices[0].delta.content or "")
    return s

In [60]:
def create_dataset(series, p = "Introduction: I'm seeking data augmentation for causal sentences, where each sentence comprises a cause, effect, and signal./n Please adhere to the following format:/n Cause: The reason for an event happening, to be enclosed between <ARG0> and </ARG0>./n Effect: The event that occurs due to the cause, to be enclosed between <ARG1> and </ARG1>./n Signal: Words that transition the cause to the effect, to be enclosed between <SIG0> and </SIG0>./n for example:/n  1. <ARG0>The heavy rainstorm</ARG0> in the evening <sig0>caused</sig0> <ARG1>widespread flooding in the city</ARG1>./n 2. After the long hours of practice, <ARG0>the team's dedication</ARG0> <sig0>led to</sig0> <ARG1>a remarkable improvement in their performance</ARG1>./n 3. <ARG0>The sudden power outage</ARG0> during the storm <sig0>resulted in</sig0> <ARG1>a disruption of the entire communication system</ARG1>./n 4. (Amelia's convincing arguments <sig0>played a crucial role in</sig0> <ARG0>convincing the board</ARG0> <ARG1>to approve the new project proposal</ARG1>./n  Augment the following sentences using the above format\n"):
    s = ""
    for i in range(0, len(series), 3):
        si = ""
        subset = series[i:i+3]
        si = create_subdataset(subset, p)
        s = s + si
    return s

In [61]:
def create_dataset_s(series, p = "Introduction: I'm seeking data augmentation for causal sentences, where each sentence comprises a cause, effect, and signal./n Please adhere to the following format:/n Cause: The reason for an event happening, to be enclosed between <ARG0> and </ARG0>./n Effect: The event that occurs due to the cause, to be enclosed between <ARG1> and </ARG1>./n Signal: Words that transition the cause to the effect, to be enclosed between <SIG0> and </SIG0>./n for example:/n  1. <ARG0>The heavy rainstorm</ARG0> in the evening <sig0>caused</sig0> <ARG1>widespread flooding in the city</ARG1>./n 2. After the long hours of practice, <ARG0>the team's dedication</ARG0> <sig0>led to</sig0> <ARG1>a remarkable improvement in their performance</ARG1>./n 3. <ARG0>The sudden power outage</ARG0> during the storm <sig0>resulted in</sig0> <ARG1>a disruption of the entire communication system</ARG1>./n 4. (Amelia's convincing arguments <sig0>played a crucial role in</sig0> <ARG0>convincing the board</ARG0> <ARG1>to approve the new project proposal</ARG1>./n  Augment the following sentences using the above format\n"):
    s = ""
    f = True
    for i in range(0, len(series)):
        print('<here ' + str(i) + '>')
        si = ""
        subset = series.loc[i]
        print("< " + subset + " >")
        si = create_subdataset(subset, f, p)
        print('( ' + si + ' )')
        s = s + '\n' + si
    return s

In [27]:
test1 = create_dataset_s(h_text)
print(test1)

<here 0>
< The State alleged they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016 , allegedly over the allocation of low cost ( RDP ) houses at Marikana West Extension 2 . >
( 1. <ARG0>The dispute over the allocation of low-cost (RDP) houses at Marikana West Extension 2</ARG0> <sig0>led to</sig0> <ARG1>the State allegedly hacking Sabata Petros Chale, 39, to death in Marikana West on December 8, 2016</ARG1>.
2. On December 8, 2016, <ARG0>an alleged disagreement regarding the allocation of low-cost (RDP) houses at Marikana West Extension 2</ARG0> <sig0>resulted in</sig0> <ARG1>the State hacking Sabata Petros Chale, 39, to death in Marikana West</ARG1>.
3. <ARG0>The alleged controversy surrounding the distribution of low-cost (RDP) houses at Marikana West Extension 2</ARG0> <sig0>caused</sig0> <ARG1>the State to allegedly hack Sabata Petros Chale, 39, to death on December 8, 2016</ARG1> in Marikana West.
4. <ARG0>The State's accusation of hacking Sabata 

In [28]:
test2 = create_dataset(h_text)
print(test2)

1. <ARG0>The State's allegation</ARG0> <sig0>claimed</sig0> <ARG1>that they hacked Sabata Petros Chale, 39, to death in Marikana West on December 8, 2016</ARG1>, allegedly over the allocation of low-cost (RDP) houses at Marikana West Extension 2.

2. <ARG0>A group of about 30 people</ARG0> <sig0>chased</sig0> <ARG1>Chale</ARG1> and <sig0>hacked him to death with pangas, axes, and spears</sig0>.

3. <ARG0>The farmworkers</ARG0> <sig0>resumed their strike on Tuesday</sig0> when <sig0>their demands were not met</sig0>.1. <ARG0>Demonstrators</ARG0> <sig0>have filed for a permit to hold a rally on Saturday</sig0> <ARG1>in Yuen Long</ARG1>, the district on the outskirts of Hong Kong where dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods, leaving at least 45 people in hospital.

2. <ARG0>Footage of the attack</ARG0>, which included <sig0>a pregnant woman being hit</sig0>, <sig0>protesters being punched and kneed</sig0>, and <sig0>commuters screami

In [29]:
h_text.loc[0]

'The State alleged they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016 , allegedly over the allocation of low cost ( RDP ) houses at Marikana West Extension 2 .'

In [30]:
combined_text = '\n\n'.join(h_text)
print(combined_text)

The State alleged they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016 , allegedly over the allocation of low cost ( RDP ) houses at Marikana West Extension 2 .

Chale was allegedly chased by a group of about 30 people and was hacked to death with pangas , axes and spears .

The farmworkers ' strike resumed on Tuesday when their demands were not met .

Demonstrators have filed for a permit to hold a rally on Saturday in Yuen Long , the district on the outskirts of Hong Kong where dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods , leaving at least 45 people in hospital .

Footage of the attack , which included a pregnant woman being hit , protesters being punched and kneed , and commuters screaming and crying while trying to shield themselves , emerged on Sunday night , fuelling further political unrest as demonstrators , opposition lawmakers and others demanded answers from authorities for failing to stop 

In [62]:
texts_wp = df['causal_text_w_pairs']
h_text2 = texts_wp.head(10)

In [63]:
texts_wp.head(10)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  ['The State alleged <ARG1>they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016</ARG1> , allegedly <SIG0>over</SIG0> <ARG0>the allocation of low cost ( RDP ) houses at Marikana West Extension 2</ARG0> .']
1                                                                                                                                                                                                     

In [32]:
test1 = create_dataset_s(h_text2)
print(test1)

<here 0>
< ['The State alleged <ARG1>they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016</ARG1> , allegedly <SIG0>over</SIG0> <ARG0>the allocation of low cost ( RDP ) houses at Marikana West Extension 2</ARG0> .'] >
( 1. <ARG0>The alleged argument over the allocation of low-cost (RDP) houses at Marikana West Extension 2</ARG0> allegedly <SIG0>led to</SIG0> <ARG1>the State accusing them of hacking Sabata Petros Chale, 39, to death in Marikana West on December 8, 2016</ARG1>.

2. <ARG0>The alleged dispute regarding the allocation of low-cost (RDP) houses at Marikana West Extension 2</ARG0> allegedly <SIG0>resulted in</SIG0> <ARG1>the State claiming that they hacked Sabata Petros Chale, 39, to death in Marikana West on December 8, 2016</ARG1>.

3. <ARG0>The State's allegation of hacking Sabata Petros Chale, 39, to death in Marikana West on December 8, 2016</ARG0> allegedly <SIG0>stemmed from</SIG0> <ARG1>a reported disagreement over the allocation of lo

In [36]:
df.head(20)

,corpus,doc_id,sent_id,eg_id,index,text,causal_text_w_pairs,num_rs
0,cnc,train_01_0,892,0,cnc_train_01_0_892_0,"The State alleged they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016 , allegedly over the allocation of low cost ( RDP ) houses at Marikana West Extension 2 .","['The State alleged <ARG1>they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016</ARG1> , allegedly <SIG0>over</SIG0> <ARG0>the allocation of low cost ( RDP ) houses at Marikana West Extension 2</ARG0> .']",1
1,cnc,train_01_1,2714,0,cnc_train_01_1_2714_0,"Chale was allegedly chased by a group of about 30 people and was hacked to death with pangas , axes and spears .",[],0
2,cnc,train_01_10,2619,0,cnc_train_01_10_2619_0,The farmworkers ' strike resumed on Tuesday when their demands were not met .,"[""<ARG1>The farmworkers ' strike resumed on Tuesday</ARG1> when <ARG0>their demands were not met</ARG0> .""]",1
3,cnc,train_01_100,2680,0,cnc_train_01_100_2680_0,"Demonstrators have filed for a permit to hold a rally on Saturday in Yuen Long , the district on the outskirts of Hong Kong where dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods , leaving at least 45 people in hospital .","['<ARG1>Demonstrators have filed for a permit</ARG1> <ARG0><SIG0>to</SIG0> hold a rally on Saturday in Yuen Long</ARG0> , the district on the outskirts of Hong Kong where dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods , leaving at least 45 people in hospital .', 'Demonstrators have filed for a permit to hold a rally on Saturday in Yuen Long , the district on the outskirts of Hong Kong where <ARG0>dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods</ARG0> , <ARG1><SIG0>leaving</SIG0> at least 45 people in hospital</ARG1> .']",2
4,cnc,train_01_101,3090,0,cnc_train_01_101_3090_0,"Footage of the attack , which included a pregnant woman being hit , protesters being punched and kneed , and commuters screaming and crying while trying to shield themselves , emerged on Sunday night , fuelling further political unrest as demonstrators , opposition lawmakers and others demanded answers from authorities for failing to stop the violence .","['<ARG0>Footage of the attack , which included a pregnant woman being hit , protesters being punched and kneed , and commuters screaming and crying while trying to shield themselves , emerged on Sunday night</ARG0> , <ARG1><SIG0>fuelling</SIG0> further political unrest</ARG1> as demonstrators , opposition lawmakers and others demanded answers from authorities for failing to stop the violence .', 'Footage of the attack , which included a pregnant woman being hit , protesters being punched and kneed , and commuters screaming and crying while trying to shield themselves , emerged on Sunday night , fuelling further political unrest as <ARG1>demonstrators , opposition lawmakers and others demanded answers from authorities</ARG1> <SIG0>for</SIG0> <ARG0>failing to stop the violence</ARG0> .']",2
5,cnc,train_01_102,274,0,cnc_train_01_102_274_0,"Max Chung , who delivered the application for a letter of no objection from the police , that is required to hold a rally , said : “ We want to show to the public and international community that Hong Kongers , we will never surrender in front of terrorism … To show our solidarity and say no to terrorism , we have to stand up. ” Chung said : “ Yuen Long was under terrorist attack and we have no choice but to take it back. ” The protests , principally over a controversial bill that would allow suspects to be extradited from Hong Kong to mainland China , have taken on new demands , including an investigation into police use of rubber bullets , teargas and physical violence against protesters .",[],0
6,cnc,train_01_103,2840,0,cnc_train_01_103_2840_0,Protesters are angry at the police ’ s slow response to the attack in Yuen Long an

In [69]:
ne_df = df[df['num_rs'] != 0].reset_index(drop=True)

In [70]:
ntexts_wp = ne_df['causal_text_w_pairs']
nh_text2 = ntexts_wp.head(10)
ntexts = ne_df['text']
nh_text = ntexts.head(10)

In [71]:
ntest1 = create_dataset_s(nh_text)
print(ntest1)

<here 0>
< The State alleged they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016 , allegedly over the allocation of low cost ( RDP ) houses at Marikana West Extension 2 . >
( 1. <ARG0>The State</ARG0> alleged <SIG0>that</SIG0> <ARG1>they hacked Sabata Petros Chale, 39, to death</ARG1> in Marikana West, on December 8, 2016, allegedly over the allocation of low cost (RDP) houses at Marikana West Extension 2.

2. In Marikana West, on December 8, 2016, <ARG0>the State</ARG0> <SIG0>claimed</SIG0> <ARG1>that they hacked Sabata Petros Chale, 39, to death</ARG1> allegedly over the allocation of low cost (RDP) houses at Marikana West Extension 2.

3. Allegedly over the allocation of low cost (RDP) houses at Marikana West Extension 2, <ARG0>the State</ARG0> <SIG0>accused</SIG0> <ARG1>them of hacking Sabata Petros Chale, 39, to death</ARG1> in Marikana West, on December 8, 2016.

4. <ARG0>The State</ARG0> claimed <SIG0>that</SIG0> <ARG1>they killed Sabata Petro

In [72]:
ntest2 = create_dataset_s(nh_text2)
print(ntest2)

<here 0>
< ['The State alleged <ARG1>they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016</ARG1> , allegedly <SIG0>over</SIG0> <ARG0>the allocation of low cost ( RDP ) houses at Marikana West Extension 2</ARG0> .'] >
( 1. <ARG0>The alleged dispute over the allocation of low cost (RDP) houses at Marikana West Extension 2</ARG0> allegedly <SIG0>led to</SIG0> <ARG1>the State accusing them of hacking Sabata Petros Chale, 39, to death in Marikana West on December 8, 2016</ARG1>.

2. <ARG0>The alleged altercation regarding the allocation of low cost (RDP) houses at Marikana West Extension 2</ARG0> allegedly <SIG0>resulted in</SIG0> <ARG1>the State claiming that they hacked Sabata Petros Chale, 39, leading to his death in Marikana West on December 8, 2016</ARG1>.

3. <ARG0>The alleged conflict over the distribution of low cost (RDP) houses at Marikana West Extension 2</ARG0> allegedly <SIG0>caused</SIG0> <ARG1>the State to accuse them of hacking Sabata Petro

In [74]:
ntest3 = create_dataset(nh_text)
print(ntest3)

1. <ARG0>The State</ARG0> alleged they hacked Sabata Petros Chale, 39, to death in Marikana West, on December 8, 2016, <sig0>allegedly over</sig0> <ARG1>the allocation of low cost (RDP) houses at Marikana West Extension 2</ARG1>.

2. <ARG0>The farmworkers' strike</ARG0> <sig0>resumed</sig0> <ARG1>on Tuesday</ARG1> when their demands were not met.

3. <ARG0>Demonstrators</ARG0> <sig0>have filed for</sig0> <ARG1>a permit to hold a rally on Saturday</ARG1> in Yuen Long, the district on the outskirts of Hong Kong <sig0>where</sig0> dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods, leaving at least 45 people in hospital.1. <ARG0>The footage of the attack</ARG0> showing a pregnant woman being hit, protesters being punched and kneed, and commuters screaming and crying while trying to shield themselves <sig0>caused</sig0> <ARG1>further political unrest</ARG1> as demonstrators, opposition lawmakers, and others demanded answers from authorities for f

In [75]:
ntest4 = create_dataset(nh_text2)
print(ntest4)

1. <ARG0>The allocation of low cost ( RDP ) houses at Marikana West Extension 2</ARG0> allegedly <SIG0>led to</SIG0> <ARG1>the State alleging that they hacked Sabata Petros Chale, 39, to death in Marikana West, on December 8, 2016</ARG1>.

2. <ARG0>The farmworkers' demands</ARG0> <SIG0>led to</SIG0> <ARG1>the resumption of their strike on Tuesday</ARG1> when they were not met.

3. <ARG0>Dozens of masked men chasing and beating commuters and protesters with wooden poles and metal rods</ARG0> <SIG0>resulted in</SIG0> <ARG1>demonstrators filing for a permit to hold a rally on Saturday in Yuen Long, the district on the outskirts of Hong Kong</ARG1>, where at least 45 people were left in the hospital.

4. <ARG1>At least 45 people in hospital</ARG1> <SIG0>were caused by</SIG0> <ARG0>demonstrators filing for a permit to hold a rally on Saturday in Yuen Long, the district on the outskirts of Hong Kong</ARG0>, where dozens of masked men chased and beat commuters and protesters with wooden poles

In [ ]:
prompt = "Introduction: I'm seeking data augmentation for causal sentences, where each sentence comprises a cause, effect, and signal./n Please adhere to the following format:/n Cause: The reason for an event happening, to be enclosed between <ARG0> and </ARG0>./n Effect: The event that occurs due to the cause, to be enclosed between <ARG1> and </ARG1>./n Signal: Words that transition the cause to the effect, to be enclosed between <SIG0> and </SIG0>./n for example:/n  1. <ARG0>The heavy rainstorm</ARG0> in the evening <sig0>caused</sig0> <ARG1>widespread flooding in the city</ARG1>./n 2. After the long hours of practice, <ARG0>the team's dedication</ARG0> <sig0>led to</sig0> <ARG1>a remarkable improvement in their performance</ARG1>./n 3. <ARG0>The sudden power outage</ARG0> during the storm <sig0>resulted in</sig0> <ARG1>a disruption of the entire communication system</ARG1>./n 4. (Amelia's convincing arguments <sig0>played a crucial role in</sig0> <ARG0>convincing the board</ARG0> <ARG1>to approve the new project proposal</ARG1>./n Augment the following sentences using the above format giving only one augmented sentence per sentence given:\n"

In [77]:
ne_df.head(10)

,corpus,doc_id,sent_id,eg_id,index,text,causal_text_w_pairs,num_rs
0,cnc,train_01_0,892,0,cnc_train_01_0_892_0,"The State alleged they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016 , allegedly over the allocation of low cost ( RDP ) houses at Marikana West Extension 2 .","['The State alleged <ARG1>they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016</ARG1> , allegedly <SIG0>over</SIG0> <ARG0>the allocation of low cost ( RDP ) houses at Marikana West Extension 2</ARG0> .']",1
1,cnc,train_01_10,2619,0,cnc_train_01_10_2619_0,The farmworkers ' strike resumed on Tuesday when their demands were not met .,"[""<ARG1>The farmworkers ' strike resumed on Tuesday</ARG1> when <ARG0>their demands were not met</ARG0> .""]",1
2,cnc,train_01_100,2680,0,cnc_train_01_100_2680_0,"Demonstrators have filed for a permit to hold a rally on Saturday in Yuen Long , the district on the outskirts of Hong Kong where dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods , leaving at least 45 people in hospital .","['<ARG1>Demonstrators have filed for a permit</ARG1> <ARG0><SIG0>to</SIG0> hold a rally on Saturday in Yuen Long</ARG0> , the district on the outskirts of Hong Kong where dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods , leaving at least 45 people in hospital .', 'Demonstrators have filed for a permit to hold a rally on Saturday in Yuen Long , the district on the outskirts of Hong Kong where <ARG0>dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods</ARG0> , <ARG1><SIG0>leaving</SIG0> at least 45 people in hospital</ARG1> .']",2
3,cnc,train_01_101,3090,0,cnc_train_01_101_3090_0,"Footage of the attack , which included a pregnant woman being hit , protesters being punched and kneed , and commuters screaming and crying while trying to shield themselves , emerged on Sunday night , fuelling further political unrest as demonstrators , opposition lawmakers and others demanded answers from authorities for failing to stop the violence .","['<ARG0>Footage of the attack , which included a pregnant woman being hit , protesters being punched and kneed , and commuters screaming and crying while trying to shield themselves , emerged on Sunday night</ARG0> , <ARG1><SIG0>fuelling</SIG0> further political unrest</ARG1> as demonstrators , opposition lawmakers and others demanded answers from authorities for failing to stop the violence .', 'Footage of the attack , which included a pregnant woman being hit , protesters being punched and kneed , and commuters screaming and crying while trying to shield themselves , emerged on Sunday night , fuelling further political unrest as <ARG1>demonstrators , opposition lawmakers and others demanded answers from authorities</ARG1> <SIG0>for</SIG0> <ARG0>failing to stop the violence</ARG0> .']",2
4,cnc,train_01_103,2840,0,cnc_train_01_103_2840_0,Protesters are angry at the police ’ s slow response to the attack in Yuen Long and their pursuit of the case .,['<ARG1>Protesters are angry</ARG1> <SIG0>at</SIG0> <ARG0>the police ’ s slow response to the attack in Yuen Long and their pursuit of the case</ARG0> .'],1
5,cnc,train_01_104,2950,0,cnc_train_01_104_2950_0,The violence in Yuen Long has heightened tensions and fears of further attacks .,['<ARG0>The violence in Yuen Long</ARG0> <SIG0>has <ARG1>heightened</SIG0> tensions and fears of further attacks</ARG1> .'],1
6,cnc,train_01_107,3187,0,cnc_train_01_107_3187_0,"On Tuesday , a group of aviation staff called for a protest at Hong Kong airport on Friday to condemn the government and police for “ ignoring the random attacks on citizens in Yuen Long ” .","['On Tuesday , <ARG1>a group of aviation staff called for a protest at Hong Kong airport on Friday</ARG1> <ARG0><SIG0>to</SIG0> condemn the government and police</ARG0> for “ ignoring the random attacks on citizens in Yuen Long ” .', 

In [78]:
n2test1 = create_dataset_s(nh_text, prompt)
print(n2test1)

<here 0>
< The State alleged they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016 , allegedly over the allocation of low cost ( RDP ) houses at Marikana West Extension 2 . >
( Cause: <ARG0>The State</ARG0> alleged <sig0>that they hacked Sabata Petros Chale, 39, to death in Marikana West, on December 8, 2016</sig0>.
Effect: This alleged action took place <sig0>allegedly over the allocation of low cost (RDP) houses at Marikana West Extension 2</sig0>.
Signal: <SIG0>According to</SIG0> the State, )
<here 1>
< The farmworkers ' strike resumed on Tuesday when their demands were not met . >
( 1. <ARG0>The farmworkers' frustration</ARG0> <sig0>led to</sig0> <ARG1>a continuation of the strike on Tuesday after their demands were not met.</ARG1>
2. <ARG0>The farmworkers' disappointment</ARG0> <sig0>resulted in</sig0> <ARG1>the strike resuming on Tuesday due to their demands not being met.</ARG1>
3. <ARG0>The farmworkers' determination</ARG0> <sig0>caused</sig0>

In [88]:
n2test2 = create_dataset_s(nh_text2, prompt)
print(n2test2)

<here 0>
< ['The State alleged <ARG1>they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016</ARG1> , allegedly <SIG0>over</SIG0> <ARG0>the allocation of low cost ( RDP ) houses at Marikana West Extension 2</ARG0> .'] >
( 1. <ARG0>The alleged disagreement over the allocation of low cost ( RDP ) houses at Marikana West Extension 2</ARG0> allegedly <sig0>resulted in</sig0> <ARG1>the State accusing them of hacking Sabata Petros Chale, 39, to death in Marikana West on December 8, 2016</ARG1>.
 )
<here 1>
< ["<ARG1>The farmworkers ' strike resumed on Tuesday</ARG1> when <ARG0>their demands were not met</ARG0> ."] >
( 1. <ARG0>The failure to meet their demands</ARG0> <sig0(triggered)</sig0> <ARG1>the resumption of the farmworkers' strike on Tuesday</ARG1>.
2. <ARG0>In response to their unmet demands</ARG0>, the farmworkers decided to <sig0>resume</sig0> <ARG1>their strike on Tuesday</ARG1>.
3. <ARG0>When their demands went unaddressed</ARG0>, the farmworkers <

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-8MnE3IjMvdwlgS6ynp57DWFI on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [91]:
n2test3 = create_dataset(nh_text, prompt)
print(n2test3)

1. <ARG0>The State</ARG0> alleged <sig0>that</sig0> they hacked Sabata Petros Chale, 39, to death in Marikana West, on December 8, 2016, allegedly over the allocation of low-cost (RDP) houses at Marikana West Extension 2 <ARG1>resulting in a tragic loss of life</ARG1>.

2. <ARG0>The farmworkers</ARG0> <sig0,resumed their strike on Tuesday after</sig0> their demands were not met <ARG1>leading to a continuation of the labor unrest</ARG1>.

3. <ARG0>Demonstrators</ARG0> have filed for a permit to hold a rally on Saturday in Yuen Long, the district on the outskirts of Hong Kong <sig0>where</sig0> dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods, leaving at least 45 people in hospital <ARG1,triggering widespread concern and the need for a public demonstration</ARG1>.1. The slow response of the police <sig0>has enraged</sig0> protesters, which resulted in <ARG1>heightened tensions and fears of further attacks</ARG1>.

2. The police's pursuit of t

In [92]:
n2test4 = create_dataset(nh_text2, prompt)
print(n2test4)

1. <ARG0>The allocation of low cost ( RDP ) houses at Marikana West Extension 2</ARG0> allegedly <SIG0>led to</SIG0> <ARG1>the State alleging that they hacked Sabata Petros Chale, 39, to death in Marikana West, on December 8, 2016</ARG1>.

2. When <ARG0>their demands were not met</ARG0>, <SIG0>the farmworkers' strike resumed on Tuesday</SIG0> <ARG1>again</ARG1>.

3. <ARG0>Dozens of masked men chasing and beating commuters and protesters with wooden poles and metal rods</ARG0> <SIG0>prompted</SIG0> <ARG1>the demonstrators to file for a permit to hold a rally on Saturday in Yuen Long</ARG1>, the district on the outskirts of Hong Kong, where at least 45 people were left in hospital.

4. <ARG0>Amelia's convincing arguments</ARG0> <SIG0>played a crucial role in</SIG0> <ARG1>convincing the board to approve the new project proposal</ARG1>, according to their remarks.1. <ARG0>Footage of the attack , which included a pregnant woman being hit , protesters being punched and kneed , and commuters 

In [84]:
def extract_between_tags_regex(input_string, tag):
    pattern = re.compile(f"<{tag}>(.*?)</{tag}>")
    match = pattern.search(input_string)
    
    if match:
        result = match.group(1)
        return result
    else:
        return None

In [89]:
def conv_s(sentence):
    cause = extract_between_tags_regex(sentence, 'ARG0')
    effect = extract_between_tags_regex(sentence, 'ARG1')
    signal = extract_between_tags_regex(sentence, 'SIG0')
    s = "For example: /n" + sentence + "/nWhere the cause is " + cause + "/nThe effect is " + effect + "/nThe signal is " + signal
    return s

In [90]:
tss = nh_text2.loc[0]
tssss = conv_s(tss)
print(tssss)

For example: /n['The State alleged <ARG1>they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016</ARG1> , allegedly <SIG0>over</SIG0> <ARG0>the allocation of low cost ( RDP ) houses at Marikana West Extension 2</ARG0> .']/nWhere the cause is the allocation of low cost ( RDP ) houses at Marikana West Extension 2/nThe effect is they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016/nThe signal is over
